# Audio Digital Signal Processing - EDA  
<p>
A foundational guide for manipulating and interpreting audio files
</p>

--- 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.style as style 
from matplotlib import cm
style.use('ggplot')
sns.set_context('notebook')
cmap = 'RdYlGn'
sns.set_palette(cmap)

import struct
from glob import glob
import librosa as lr
import librosa.display

from mpl_toolkits.mplot3d import axes3d
import IPython.display as ipd

%matplotlib inline

___
___
### Individual clip analysis

In [3]:
clips = [x for x in glob('./audio_files/*.wav')]

In [4]:
amp, sr = lr.load(clips[-1])

In [ ]:
plt.figure(figsize=(14,7))
lr.display.waveshow(amp, sr=sr, alpha=0.7, color='#7a1b4f', x_axis='s')
plt.ylabel('Amplitude (B)')
plt.xlabel('Time (s)')
plt.title(clips[-1].split('/')[-1][:-4], fontsize=32)

ipd.Audio(clips[-1])

/Users/dom/Repos/DSP_Songs/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: waveshow() keyword argument 'x_axis' has been renamed to 'axis' in version 0.10.0.
	This alias will be removed in version 1.0.
  


In [ ]:
clips

In [ ]:
track = clips[-1]

amp, sr = lr.load(track)

_, ax = plt.subplots(nrows=2, ncols=2, figsize=(24,14))
lr.display.waveshow(amp, sr=sr, alpha=0.85, color='#7a1b4f', x_axis='s', ax=ax[0,0])
ax[0,0].set_ylabel('Amplitude (B)')
# ax1.set_xlabel('Time (s)')
ax[0,0].set_title(track.split('/')[-1][:-4] + ' || Waveform', fontsize=22)
ax[0,0].grid(linestyle='-', linewidth='0.5', color='#cccccc')


X = librosa.stft(amp)  # , n_fft=)
Xdb = librosa.amplitude_to_db(abs(X))

librosa.display.specshow(Xdb, sr=sr, x_axis='s', y_axis='hz', cmap='magma', ax=ax[1,0]) 
#If to pring log of frequencies  
ax[1,0].set_title(track.split('/')[-1][:-4] + ' || Spectrogram', fontsize=22)

# plt.colorbar()
ax[1,0].grid(linestyle='-', linewidth='0.5', color='#cccccc')

librosa.display.specshow(Xdb, sr=sr, x_axis='s', y_axis='log', cmap='magma', ax=ax[1,1])
ax[1,1].set_title(track.split('/')[-1][:-4] + ' || Logged Spectrogram', fontsize=22)

ax[0,1].set_visible(False)
ipd.Audio(track)

In [ ]:
audio, sr = lr.load(clips[4])

D = librosa.stft(audio)  # , n_fft=)
S = librosa.amplitude_to_db(abs(D))
# Dimensions of specrogram D - (f, t)
# t: frames_to_samples
# f: fft_frequencies
# np.abs(D[..., f, t]) is the magnitude of frequency bin f at frame t, and
# np.angle(D[..., f, t]) is the phase of frequency bin f at frame t

S = np.swapaxes(np.swapaxes(S[:-1], 0, 1)[:-1], 0, 1)
ts = np.arange(0, S.shape[1])  # .reshape((1,-1))
freq = np.linspace(0, sr*S.shape[0], S.shape[0]).reshape((-1,1))

print(f'ts: {ts.shape}\nfreq: {freq.shape}')
np.broadcast_shapes(ts.shape, freq.shape, S.shape)

In [ ]:
import plotly.graph_objects as go

sh_0, sh_1 = S.shape
x, y = np.linspace(0, S.shape[1], S.shape[1]), np.linspace(0, sr*S.shape[0], S.shape[0])
fig = go.Figure(data=[go.Surface(z=S, x=x, y=y)])
fig.update_layout(title='Clip name - ADD drop', autosize=False,
                  scene ={            
                    "xaxis": {"nticks": 11, "title": "time"},
                    "yaxis": {"title": "frequency"},
                    "zaxis": {"nticks": 10},
                    'camera_eye': {"x": 0, "y": -1, "z": 1},
                    "aspectratio": {"x": 0.8, "y": 0.6, "z": 0.4}
                  },
                    # dict(
                    # xaxis_title='time',
                    # yaxis_title='Frequency (Hz)',
                    # zaxis_title='Magnitude (dB'),),
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()
ipd.Audio(clips[-1])